<div style="text-align: center; font-weight: bold; font-size: 300%">Projet MODAL INF473-V 

</div>                                                        <br /> 
<div style="text-align: center; font-weight: bold; font-size: 180%"> Création des datasets à partir des images augmentés </div>                                                        <br />  
<div style="text-align: center; font-size: 150%">École Polytechnique, juin 2020</div><br />  
<div style="text-align: center; font-size: 120%"> Youssef Allouah et Paul Calot </div>



In [0]:
import pandas as pd # to read csv files
import numpy as np
import imageio # to read imported img
import torch
from torchvision import datasets, transforms # to transforms to tensor etc. and create our own datasets
import matplotlib.pyplot as plt # to show img
import tqdm

# Importing images from git

In [0]:
# reading the initial data from the gitlab
!git clone https://gitlab.binets.fr/paul.calot-plaetevoet/plantpathology_2020.git 

Cloning into 'plantpathology_2020'...
remote: Enumerating objects: 133095, done.
remote: Counting objects: 100% (133095/133095), done.
remote: Compressing objects: 100% (132919/132919), done.
remote: Total 133095 (delta 365), reused 132904 (delta 174), pack-reused 0
Receiving objects: 100% (133095/133095), 2.03 GiB | 32.51 MiB/s, done.
Resolving deltas: 100% (365/365), done.
Checking out files: 100% (35002/35002), done.


# Saving the dataset in google drive

Because *git* compresses data, we can't save the *torch.datasets* here. We thus used *google drive*. 


In [0]:
# if you are on google.colab :

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# my drive ...
root_dir = "/content/gdrive/My Drive/projet - MODAL/"



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Definition of a few functions useful to convert our initial images

In [0]:
to_pil = transforms.ToPILImage()

def imshow(img_Tensor):
  img = to_pil(img_Tensor)
  plt.imshow(img,cmap=plt.cm.gray)
  plt.show()


def uint8_RGB_to_float32(img):
  return img.astype(np.float32)/255.0


## Images to Tensors functions

In *get_y_tensor*, the lines :


```
if(tt>1):
      tt = tt -1
```
are only useful for 3 classes datasets. They convert 3 to 2 and 2 to 1 for classe number as we deleted the *multiple diseases" class.
You should comment those lines if you are creating a 4-classes dataset.


In [0]:

    # defining functions for later
def get_y_tensor(start, end, y_train): # takes indexes -> returns a tensor
  y_train_tensor = []

  for k in range(start, end):
    tt = np.argmax(y_train[k])
    #only useful when creating a dataset for the 3-classes model
      # commentate it when creating for 4 classes
    if(tt>1):
      tt = tt -1
    
    y_train_tensor.append(tt)
    
  return torch.Tensor(np.array(y_train_tensor))
# returns a tensor

def get_x_tensor(start, end, path): 
  x_train = []
  count = start
  for k in range(start,end):
    path_img_training_k = path + str(count) + ".jpg"
    img = uint8_RGB_to_float32(imageio.imread(path_img_training_k))

    x_train.append(np.transpose(img, (2,0,1))) # may work better
    count +=1
  return torch.Tensor(x_train)

In [0]:
y_train_ = pd.read_csv("plantpathology_2020/train_3classes.csv", delimiter = ',', header=None)
y_train_

,0,1,2,3,4
0,image_id,healthy,multiple_diseases,rust,scab
1,Train_0,0,0,0,1
2,Train_1,0,0,0,1
3,Train_2,0,0,0,1
4,Train_3,1,0,0,0
...,...,...,...,...,...
3992,Train_3991,0,0,1,0
3993,Train_3992,0,0,1,0
3994,Train_3993,0,0,1,0
3995,Train_3994,0,0,1,0


# Initialization

In [0]:

y_train_ = pd.read_csv("plantpathology_2020/train_3classes.csv", delimiter = ',').values # to convert it to numpy
print("Ground truth number in the csv : " + str(len(y_train_)))
N= len(y_train_) # new dataset
m = 0 # if we want to split the dataset in two (for RAM reasons or for testing / training)
y_train = np.zeros((N,4))
for i in range(0,N):
  for j in range(0,4):
    y_train[i][j] = y_train_[i][j+1] 

path = "plantpathology_2020/images_3classes/"
name_train = "Train_"

print(y_train)
#%% creating training data set


Ground truth number in the csv : 3996
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


### Ground Truths

In [0]:
  # y train

y_train_tensor = get_y_tensor(m,N,y_train)
print("Computing y_train ... [OK]")




Computing y_train ... [OK]


In [0]:
print("Checking that we have the right amount of data and that the ground truths are the right ones : ")
print(y_train_tensor)
print(len(y_train_tensor))

Checking that we have the right amount of data and that the ground truths are the right ones : 
tensor([2., 2., 2.,  ..., 1., 1., 1.])
3996


### Features

In [0]:
x_train = get_x_tensor(m,N, path+name_train)
x_train = torch.Tensor(x_train)
print("DONE !")


DONE !


## Creating the dataset

In [0]:
augmented_testing_data = torch.utils.data.TensorDataset(x_train,y_train_tensor) # converting only in the end to a Tensor seems to work; but it also takes a lot of RAM.

In [0]:
del y_train_tensor
del x_train

In [0]:
torch.save(augmented_testing_data,"training_set_3classes")

In [0]:
del augmented_testing_data